## Import

In [5]:
import pandas as pd
import numpy as np
import random

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Tag Setting

In [6]:
taglist = pd.read_csv('taglist.csv')
taglist

,"('기본', '품질')","('기본', '작화')","('기본', '테마')","('기본', '초점')","('기본', '연출')","('기본', '배경')","('기본', '장소')","('작품', '작품')","('작품', '캐릭터')","('인물', '인원')",...,"('동작 2', '브래지어')","('동작 2', '팬티')","('동작 2', '양말')","('동작 2', '모자')","('동작 2', '안경')","('동작 2', '마스크')","('동작 2', '넥타이')","('동작 2', '귀걸이')","('행동', '동사')","('행동', '동명사')"
0,extra_legs,manga_photorealistic,City_Pop,feet_out_of_frame,multiple_4koma,streaked,megastructure,Sword_Art_Online,White_Fox,6others,...,bra_on_ass,panties_on_head,taut_legwear,backwards_hat,eyewear_view,holding_mask,necktie_grab,adjusting_earrings,waving,shaking
1,missing_hips,Graphic_Novel,graphite,thigh_focus,1koma,striped,lake,Danganronpa,Artoria_Pendragon,3girls,...,bra_removed,holding_panties,mismatched_legwear,sideways_hat,eyewear_hang,mask_on_head,NaN,NaN,splashing,thinking
2,missing_nipples,Cartoon_realistic,Funk,looking_down,2koma,sky_blue,locker_room,MAPPA_Corporation,Orsola_Aquinas,1boy,...,bra_pull,adjusting_panties,torn_legwear,hat_over_one_eye,eyewear_on_head,removing_mask,NaN,NaN,slipping,scratching
3,missing_chest,Cartoon_photorealistic,Pop,ass_focus,borderless_panels,print,convention,Fairy_Tail,Itsuwa,2girls,...,bra_lift,panties_aside,wet_legwear,adjusting_headwear,looking_over_eyewear,mask_removed,NaN,NaN,bumping,slashing
4,missing_eyes,Monochrome,Adult_Video,hip_focus,3koma,blue,volcano,Pokemon,Artoria_Caster,1girl,...,holding_bra,hand_in_panties,removing_legwear,hat_over_eyes,eyewear_on_headwear,mask_over_one_eye,NaN,NaN,elbowing,imagining
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Calamity_Jane,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sword_Maiden,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jaguar_Man,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mito_Uzumaki,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


일단은 6가지의 대표적인 특징을 선택함   
사람은 여자만 있는 걸로. Novel AI에 학습된 남자 데이터는 보통 게이(...) 그림이 많아서 학습이 어려움   
머리 색, 눈동자 색은 적당한 색깔로 설정한 다음, 결과를 보며 수정할 생각   
배경, 의상, 체형은 가짓수가 많아서 내가 직접 보기에 괜찮은 것만 추려서 넣음   
중첩 가능한 태그인 others도 마찬가지.

## Make Tag

In [7]:
tagLength = 10
def get_tag(tagLength=tagLength, tagLists=taglist) -> str:
    '''
    태그 모음에서 랜덤하게 태그를 뽑아 태그 조합을 생성하는 함수입니다. 
    tagLength는 태그의 개수입니다.
    tagLength에서 others를 제외한 범주의 개수를 뺀 만큼이 중첩 가능한 태그의 개수입니다.
    masterpiece, best quality는 무조건 포함됩니다.
    '''

    result = 'masterpiece, best quality, '
    # 중첩 불가능한 태그 뽑기
    for i in range(len(tagLists) - 1):
        result += random.choice(tagLists[i]) + ', '

    # 중첩 가능한 태그 뽑기
    used = []
    for i in range(tagLength - len(tagLists) + 1):
        while True:
            tag = random.choice(tagLists[-1])
            if tag not in used:
                used.append(tag)
                break
        result += tag + ', '
    result = result[:-2]
    
    return result

## Get Image

In [8]:
URL = 'http://127.0.0.1:7860'
driver = webdriver.Edge(executable_path='msedgedriver.exe')
driver.get(url=URL)

# Novel AI 이외의 모델을 사용하려면
# 모델을 선택한 뒤 다음 셀 실행

/var/folders/r3/7n1n1h3x3hvcyqd0xnpjm0lr0000gn/T/ipykernel_10470/2315313369.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(executable_path='msedgedriver.exe')


WebDriverException: Message: 'msedgedriver.exe' executable needs to be in PATH. Please download from https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/


In [ ]:
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '#my-element'))
)

buttonGen = driver.find_element(By.CSS_SELECTOR,
    'button#txt2img_generate.gr-button.gr-button-lg.gr-button-primary')


In [ ]:
driver.close()